# Imports
- Import libraries
- Import data to df
    - EDA
    - check for NA's
        - NA in rating column, will be dropping
    - check for duplicated
        - No duplicates
- Split dataframe into high_rating(>=9) and low_ratings(<=4)
- NLTK and SpaCy for preprocessing text
    - create nlp object
        - Disable recognizer
    - create several versions of the data
        - lemmatized
        - tokenized
        - lemmatized-joined
        - tokenized-joined    
    - remove stopwords
    - tokenization
    - lemmantization
- Calling word cloud to see to distrubtion

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import spacy

from nltk.probability import FreqDist
from wordcloud import WordCloud

pd.set_option('display.max_colwidth', None)

In [3]:
data = "Data-NLP/movie_reviews_v2.csv"
df=pd.read_csv(data)
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8650 entries, 0 to 8649
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   review_id       8650 non-null   object 
 1   movie_id        8650 non-null   int64  
 2   imdb_id         8650 non-null   object 
 3   original_title  8650 non-null   object 
 4   review          8650 non-null   object 
 5   rating          7454 non-null   float64
dtypes: float64(1), int64(1), object(4)
memory usage: 405.6+ KB


,review_id,movie_id,imdb_id,original_title,review,rating
0,64ecc16e83901800af821d50,843,tt0118694,花樣年華,"This is a fine piece of cinema from Wong Kar-Wai that tells us a story of two people whom circumstance throws together - but not in a way you might expect. We start with two couples who move into a new building. One a newspaper man with his wife, the other a business executive and his wife. The businessman is rarely at home, nor is the journalist's wife - which leaves the other two at increasingly loose ends. It isn't long before a friendship develops - usually over some noodles - and though entirely platonic, their relationship becomes solid and trusting. Then, something entirely unexpected occurs that causes the two to refocus their relationship in the face of betrayal and disappointment - and well.... What helps this film stand out are the performances from both Tony Leung (""Chow Mo-wan"") and the gorgeous Maggie Cheung (""Mrs. Chan"") - it's a love story, and it burns intensely - but not physically - as their story subsumes both themselves and the audience. The film looks marvellous, is paced stylishly and uses quite a bit of Nat King Cole to set and keep the mood atmospheric, romantic and interesting. The photography offers us lingering close ups, we get a sense of them falling in love as if we, too, were undergoing the same experience and that all adds to the richness of the story. To give us a breather from the intensity, there are some fun scenes with their very close neighbours who love nothing better than to sit up all night playing mahjong!! Super film, this - flew by.",7.0
1,57086ff5c3a3681d29001512,7443,tt0120630,Chicken Run,"A guilty pleasure for me personally, as I love both 'The Great Escape' and most of the works I have seen, over the years, from this rightfully-esteemed British animation company. Highly recommended both for children and for adults who enjoy animation.",9.0
2,5bb5ac829251410dcb00810c,7443,tt0120630,Chicken Run,"Made my roommate who hates stop-motion animation watched this in 2018 and even he had a good time. It's maybe not as great as I remember thinking it was when I was a little kid, but it still holds up to some degree.\r\n\r\n_Final rating:★★★ - I liked it. Would personally recommend you give it a go._",6.0
3,5f0c53a013a32000357ec505,7443,tt0120630,Chicken Run,"A very good stop-motion animation!\r\n\r\n<em>'Chicken Run'</em>, which I watched a crap tonne when I was little but not for a vast number of years now, is an impressive production given it came out in 2000. Despite a pretty simple feel to the film, it's a very well developed concept.\r\n\r\nThe, admittedly short, run time truly flies by. Of course everything looks, relatively, terrific, but I'm impressed by the pacing which is clean. As for the cast, Julia Sawalha is the definite standout - she's fantastic as Ginger. Timothy Spall and Phil Daniels, as rats Nick and Fetcher, are amusing. Mel Gibson (Rocky), though still decent, isn't actually that good in this.\r\n\r\nA fun and interesting story! Definitely worth seeing if you haven't already. I'm intrigued to see where they go with the upcoming sequel.",8.0
4,64ecc027594c9400ffe77c91,7443,tt0120630,Chicken Run,"Ok, there is an huge temptation to riddle this review with puns - but I'm just going to say it's a cracking little family adventure. It's seemingly based on a whole range of classic movies from the ""Great Escape"", ""Star Trek"" to ""Love Story"" with a score cannibalised from just about any/everything ever written by Messrs. Korngold/Williams & Bernstein all adding to this really super stop-motion animation that Ray Harryhausen would have been proud of.. A flock of chickens are relentlessly exploited for their eggs by the evil ""Mrs Tweedy"" - that is, until ""Rocky"" (an on-form Mel Gibson) arrives over the fence and incites escape plans... Meantime their malevolent owner (Miranda Richardson) has plans to industrialise production - and this time, not just of the eggs..

In [4]:
df.isna().sum()

review_id            0
movie_id             0
imdb_id              0
original_title       0
review               0
rating            1196
dtype: int64

In [5]:
df.duplicated().sum()

0

In [6]:
df=df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7454 entries, 0 to 8649
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   review_id       7454 non-null   object 
 1   movie_id        7454 non-null   int64  
 2   imdb_id         7454 non-null   object 
 3   original_title  7454 non-null   object 
 4   review          7454 non-null   object 
 5   rating          7454 non-null   float64
dtypes: float64(1), int64(1), object(4)
memory usage: 407.6+ KB


In [7]:
high_rating = df[df['rating']>=9]
low_rating = df[df['rating']<=4]
high_rating.head()

,review_id,movie_id,imdb_id,original_title,review,rating
1,57086ff5c3a3681d29001512,7443,tt0120630,Chicken Run,"A guilty pleasure for me personally, as I love both 'The Great Escape' and most of the works I have seen, over the years, from this rightfully-esteemed British animation company. Highly recommended both for children and for adults who enjoy animation.",9.0
15,608e2744d8cc4a0058dc4272,2621,tt0122459,Return to Me,"Okay, I will admit I can be a bit of an easy grader when it comes to romantic comedies, as long as they are witty with likable characters, don’t insult my intelligence and have suitable happy endings (I guess if they don’t end happily, they aren’t rom-coms).\r\n\r\nI saw this movie many years ago, but had forgotten just enough of it to keep it fresh for me. Like many movies, the plot relies on a huge coincidence or three. The story gets sappy and sentimental in places, is rather predictable and has too much religion to suit me, but that last thing may suit you just fine, praise be. It was co-written and directed by the actress Bonnie Hunt, who is a supporting character in the film.\r\n\r\nBut there is a spark of chemistry between the two lead characters, the dialogue is smart and sharp, and the movie is rich with its supporting cast, such as Jim Belushi, Robert Loggia and Archie Bunker - I mean Carroll O’Connor. Plus I have liked Minnie Driver since Circle of Friends. \r\n\r\nJust keep in mind this isn’t Hitchcock or Frank Capra stuff and doesn’t try to be. Just let it flow over you",9.0
21,5570c897c3a368709c000bca,2787,tt0134847,Pitch Black,"All you people are so scared of me. Most days I'd take that as a compliment. But it ain't me you gotta worry about now.\r\n\r\nPitch Black is directed by David Twohy and collectively written by Twohy and Ken and Jim Wheat. It stars Vin Diesel, Radha Mitchell, Cole Hauser, Keith David, Lewis Fitz-Gerald, Claudia Black and Rhiana Griffith. Music is by Graeme Revell and cinematography is by David Eggby. \r\n\r\nThe year 2000 was awash with sci-fi movies, not all were stellar of course, but siting at the top end of the scale was this, Twohy's super slice of action/horror/sci-fi cake. \r\n\r\nRight from the off we are thrust on board the cargo ship Hunter-Gratzner, which as it happens is plummeting through space. The ship has been hit by meteorite debris and junior officer Caroline Fly (Mitchell), after coming out of hyper sleep, manages to crash land on a nearby planet. Only a handful of passengers survive the crash, including infamous convict Richard B. Riddick (Diesel), but as the survivors struggle to cope with the threat from within the group, it becomes apparent that there's a bigger threat soon to join the party, a deadly alien species, a species that can only operate in the dark. Pity, then, that an eclipse is due... \r\n\r\nOkies, so lets get the obvious out in the open right away, Pitch Black is a variant of Alien, one of a ream of films that grabbed the coat tails of Ridley Scott's game changer. Yet this is still a fresh movie, a lesson in low budget film making with grace, style, blood, brains and balls - the execution grade ""A"" from those involved. \r\n\r\nIt's maybe surprising given the synopsis, to find that it's a very character driven piece, with Twohy continually building his characters even as the carnage and terror is unleashed - and these are a very diverse bunch of characters. One of the universe's baddest criminals, a spunky lady officer having to take command, a morphine addicted bounty hunter, an Islamic priest and his young companions, a camp alcoholic and a couple of strays. It's the not so wild bunch, but everyone of them are afforded chance to impact on the story before and during the inevitable picking off one by one at the claws of the beasts. \r\n\r\nThe narrative strength comes via Diesel's hulking convict, he's double jointed and has had optical surgery so he can see his enemies in the dark! The group must come to rely on him for he is clea

In [10]:
nlp_no_ner = spacy.load('en_core_web_sm', disable=['parser','ner'])
nlp_no_ner.pipe_names

['tok2vec', 'tagger', 'attribute_ruler', 'lemmatizer']

# Load Preprocessor Function

In [14]:
def batch_preprocess_texts(
     texts,
     nlp=None,
     remove_stopwords=True,
     remove_punct=True,
     use_lemmas=False,
     disable=["ner"],
     batch_size=50,
     n_process=-1,):
     """Efficiently preprocess a collection of texts using nlp.pipe()

     Args:
        texts (collection of strings): collection of texts to process (e.g. df['text'])
        nlp (spacy pipe), optional): Spacy nlp pipe. Defaults to None; if None, it creates a default 'en_core_web_sm' pipe.
        remove_stopwords (bool, optional): Controls stopword removal. Defaults to True.
        remove_punct (bool, optional): Controls punctuation removal. Defaults to True.
        use_lemmas (bool, optional): lemmatize tokens. Defaults to False.
        disable (list of strings, optional): named pipeline elements to disable. Defaults to ["ner"]: Used with nlp.pipe(disable=disable)
        batch_size (int, optional): Number of texts to process in a batch. Defaults to 50.
        n_process (int, optional): Number of CPU processors to use. Defaults to -1 (meaning all CPU cores).

     Returns:
        list of tokens
     """
     # from tqdm.notebook import tqdm
     from tqdm import tqdm

     if nlp is None:
        nlp = spacy.load("en_core_web_sm")

     processed_texts = []

     for doc in tqdm(nlp.pipe(texts, disable=disable, batch_size=batch_size, n_process=n_process)):
        tokens = []
        for token in doc:
            # Check if should remove stopwords and if token is stopword
            if (remove_stopwords == True) and (token.is_stop == True):
                # Continue the loop with the next token
                continue

            # Check if should remove stopwords and if token is stopword
            if (remove_punct == True) and (token.is_punct == True):
                continue

            # Check if should remove stopwords and if token is stopword
            if (remove_punct == True) and (token.is_space == True):
                continue

            
            ## Determine final form of output list of tokens/lemmas
            if use_lemmas:
                tokens.append(token.lemma_.lower())
            else:
                tokens.append(token.text.lower())

        processed_texts.append(tokens)
     return processed_texts


In [18]:
high_rating['tokens'] = batch_preprocess_texts(high_rating['review'], nlp = nlp_no_ner)
low_rating['tokens'] = batch_preprocess_texts(low_rating['review'], nlp = nlp_no_ner)
low_rating.head()

1195it [00:10, 115.81it/s]
/var/folders/zd/1wqdzm7d7d13shmcygx_4src0000gn/T/ipykernel_1506/2012468446.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_rating['tokens'] = batch_preprocess_texts(high_rating['review'], nlp = nlp_no_ner)
1224it [00:09, 129.52it/s]
/var/folders/zd/1wqdzm7d7d13shmcygx_4src0000gn/T/ipykernel_1506/2012468446.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  low_rating['tokens'] = batch_preprocess_texts(low_rating['review'], nlp = nlp_no_ner)


,review_id,movie_id,imdb_id,original_title,review,rating,tokens
6,57b2d23dc3a36837d7000b14,955,tt0120755,Mission: Impossible II,"The first underwhelmed me, but this one straight-up bored me. Again, of course seeing Hunt climb a mountain without a harness is impressive sure. And I even quite liked the idea behind the villain of the piece (though even that angle was woefully underdeveloped).\r\n\r\nEven setting it in predominantly Australia was enough to grab my attention, and that's saying something cause I'm pretty biased when it comes to that. I hear tell that _Mission: Impossible II_ is the low point in the series, and that at least is encouraging, because if anything afterwards is much worse than this one, I don't know if I'll be able to make it through to Rogue Nation.\r\n\r\n_Final rating:★½: - Boring/disappointing. Avoid if possible._",3.0,"[underwhelmed, straight, bored, course, seeing, hunt, climb, mountain, harness, impressive, sure, liked, idea, villain, piece, angle, woefully, underdeveloped, setting, predominantly, australia, grab, attention, saying, cause, pretty, biased, comes, hear, tell, mission, impossible, ii, low, point, series, encouraging, worse, know, able, rogue, nation, final, rating:, ★, ½, boring, disappointing, avoid, possible]"
7,5e9120963679a10018375490,955,tt0120755,Mission: Impossible II,"After quite entertainign Mission Impossible I the second installment turned out ... terrible. As if the screenwriters didn't know how to fill the 2 hrs with action the overuse of heroic slow motion scenes is horrible. You almost might need a barf bag if you can't stand slo-mo every five seconds... for 15 seconds. I am amazed they did decide to push on with more M:I movies after this one, then again, these turned out much much better even if it would be hard to beat that lousy piece of crap of a movie.",2.0,"[entertainign, mission, impossible, second, installment, turned, terrible, screenwriters, know, fill, 2, hrs, action, overuse, heroic, slow, motion, scenes, horrible, need, barf, bag, stand, slo, mo, seconds, 15, seconds, amazed, decide, push, m, movies, turned, better, hard, beat, lousy, piece, crap, movie]"
20,6409202f0e441900e887a17b,4234,tt0134084,Scream 3,"**_Scream 3’s_ lackluster screenplay and unimaginative kills leave a film that is a bore to watch.**\r\n\r\nThe meta-narrative of trilogies throughout the film does not make up for how abysmal the plot was. This film creates so much lore for the past movies seemingly out of the blue, muddling up the continuity for shock value. It added very little to this film and felt forced. The entire plot point of Sidney’s mother being a former Hollywood actress was pretty lame, especially with the added plot point of her being sexually assaulted by a movie producer. It may be personal taste, but I absolutely despise when films add that plot point, as it feels incredibly hypocritical since it is something that genuinely happens to an industry, they are involved in. The love triangle between Gale, Dewey, and the Gale actress was pretty dumb and felt undeserved, as we have not seen Gale and Dewey actually together in any of the films.\r\n\r\nIn horror films, you can have a good movie with a terrible plot as long as the kills are great. Unfortunately, Scream 3 continues the trend of having boring kills with no imagination. There has been only one truly unique kill throughout the entire franchise, and the rest are unbloody stab kills. I understand the meta being about stabbing with a knife, as the in-film movie is called Stab, but please add some gore to it. There is nothing, no blood squirts or body horror, and it caused the entire movie to be boring.\r\n\r\nOur performances were exactly how I would describe Scream 2, with both Neve Campbell and Courtney Cox being the standouts and David Arquette being average at best. All of the new characters were pretty uninteresting, all of the actors in the film were annoying, and the movie producers felt generic.\r\n\r\nThe dire

In [20]:
high_rating['lemmas'] = batch_preprocess_texts(high_rating['review'], nlp = nlp_no_ner)
low_rating['lemmas'] = batch_preprocess_texts(low_rating['review'], nlp = nlp_no_ner)

1195it [00:10, 112.96it/s]
/var/folders/zd/1wqdzm7d7d13shmcygx_4src0000gn/T/ipykernel_1506/3625213148.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_rating['lemmas'] = batch_preprocess_texts(high_rating['review'], nlp = nlp_no_ner)
1224it [00:09, 127.78it/s]
/var/folders/zd/1wqdzm7d7d13shmcygx_4src0000gn/T/ipykernel_1506/3625213148.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  low_rating['lemmas'] = batch_preprocess_texts(low_rating['review'], nlp = nlp_no_ner)


In [21]:
high_rating['text-joined'] = high_rating['tokens'].map(lambda x: " ".join(x))
low_rating['text-joined'] = low_rating['tokens'].map(lambda x:' '.join(x))

/var/folders/zd/1wqdzm7d7d13shmcygx_4src0000gn/T/ipykernel_1506/2332933770.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_rating['text-joined'] = high_rating['tokens'].map(lambda x: " ".join(x))
/var/folders/zd/1wqdzm7d7d13shmcygx_4src0000gn/T/ipykernel_1506/2332933770.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  low_rating['text-joined'] = low_rating['tokens'].map(lambda x:' '.join(x))


In [22]:
high_rating['lemmas-joined'] = high_rating['lemmas'].map(lambda x: " ".join(x))
low_rating['lemmas-joined'] = low_rating['lemmas'].map(lambda x:' '.join(x))
high_rating.head()

/var/folders/zd/1wqdzm7d7d13shmcygx_4src0000gn/T/ipykernel_1506/492365132.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_rating['lemmas-joined'] = high_rating['lemmas'].map(lambda x: " ".join(x))
/var/folders/zd/1wqdzm7d7d13shmcygx_4src0000gn/T/ipykernel_1506/492365132.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  low_rating['lemmas-joined'] = low_rating['lemmas'].map(lambda x:' '.join(x))


,review_id,movie_id,imdb_id,original_title,review,rating,tokens,lemmas,text-joined,lemmas-joined
1,57086ff5c3a3681d29001512,7443,tt0120630,Chicken Run,"A guilty pleasure for me personally, as I love both 'The Great Escape' and most of the works I have seen, over the years, from this rightfully-esteemed British animation company. Highly recommended both for children and for adults who enjoy animation.",9.0,"[guilty, pleasure, personally, love, great, escape, works, seen, years, rightfully, esteemed, british, animation, company, highly, recommended, children, adults, enjoy, animation]","[guilty, pleasure, personally, love, great, escape, works, seen, years, rightfully, esteemed, british, animation, company, highly, recommended, children, adults, enjoy, animation]",guilty pleasure personally love great escape works seen years rightfully esteemed british animation company highly recommended children adults enjoy animation,guilty pleasure personally love great escape works seen years rightfully esteemed british animation company highly recommended children adults enjoy animation
15,608e2744d8cc4a0058dc4272,2621,tt0122459,Return to Me,"Okay, I will admit I can be a bit of an easy grader when it comes to romantic comedies, as long as they are witty with likable characters, don’t insult my intelligence and have suitable happy endings (I guess if they don’t end happily, they aren’t rom-coms).\r\n\r\nI saw this movie many years ago, but had forgotten just enough of it to keep it fresh for me. Like many movies, the plot relies on a huge coincidence or three. The story gets sappy and sentimental in places, is rather predictable and has too much religion to suit me, but that last thing may suit you just fine, praise be. It was co-written and directed by the actress Bonnie Hunt, who is a supporting character in the film.\r\n\r\nBut there is a spark of chemistry between the two lead characters, the dialogue is smart and sharp, and the movie is rich with its supporting cast, such as Jim Belushi, Robert Loggia and Archie Bunker - I mean Carroll O’Connor. Plus I have liked Minnie Driver since Circle of Friends. \r\n\r\nJust keep in mind this isn’t Hitchcock or Frank Capra stuff and doesn’t try to be. Just let it flow over you",9.0,"[okay, admit, bit, easy, grader, comes, romantic, comedies, long, witty, likable, characters, insult, intelligence, suitable, happy, endings, guess, end, happily, rom, coms, saw, movie, years, ago, forgotten, fresh, like, movies, plot, relies, huge, coincidence, story, gets, sappy, sentimental, places, predictable, religion, suit, thing, suit, fine, praise, co, written, directed, actress, bonnie, hunt, supporting, character, film, spark, chemistry, lead, characters, dialogue, smart, sharp, movie, rich, supporting, cast, jim, belushi, robert, loggia, archie, bunker, mean, carroll, o’connor, plus, liked, minnie, driver, circle, friends, mind, hitchcock, frank, capra, stuff, try, let, flow]","[okay, admit, bit, easy, grader, comes, romantic, comedies, long, witty, likable, characters, insult, intelligence, suitable, happy, endings, guess, end, happily, rom, coms, saw, movie, years, ago, forgotten, fresh, like, movies, plot, relies, huge, coincidence, story, gets, sappy, sentimental, places, predictable, religion, suit, thing, suit, fine, praise, co, written, directed, actress, bonnie, hunt, supporting, character, film, spark, chemistry, lead, characters, dialogue, smart, sharp, movie, rich, supporting, cast, jim, belushi, robert, loggia, archie, bunker, mean, carroll, o’connor, plus, liked, minnie, driver, circle, friends, mind, hitchcock, frank, capra, stuff, try, let, flow]",okay admit bit easy grader comes romantic comedies long witty likable characters insult intelligence suitable happy endings guess end happily rom coms saw movie years ago forgotten fresh like movies plot relies huge coincidence story gets sappy sentimental places predictable religion suit thing suit fine praise co written directed actress bonnie h

In [26]:
low_rating['lemmas'].str.contains('HTML').sum()

0.0